In [ ]:
import tensorflow as tf
from scipy.io import wavfile
import os
import scipy.signal as sps
import numpy as np
import IPython
import librosa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_path = '/content/drive/MyDrive/Models/Musics2.h5'
model = tf.keras.models.load_model(model_path)
Audios_path = "/content/drive/MyDrive/Songs"
class_names = ['Blues', 'Classical', 'Country', 'Disco', 'Hiphop', 'Jazz', 'Metal', 'Pop', 'Reggae', 'Rock']

In [ ]:
pip install ffmpeg -q   #lazem baraye pydub

In [ ]:
pip install ffprobe -q  #lazem baraye pydub

In [ ]:
pip install pydub -q  #lazem baraye pydub

In [ ]:
from pydub import AudioSegment

def make_wave_file(file):   # tabdil mp3 be wave
  sound = AudioSegment.from_file(file, format='mp3')
  sound = sound.export(file.replace(".mp3", ".wav"), format='wav')

files = os.listdir(Audios_path)
for i in files:
  if i.endswith(".mp3"):    #tabdil music ha be wave
    make_wave_file(i)

In [ ]:
audios = {}
files = os.listdir(Audios_path)
for i in files:         #load music ha dar audios
  if i.endswith(".wav") :
    audios[i], _ = librosa.load(Audios_path + '/' + i)

In [ ]:
def Voice_spec(audio) :
  spectrogram = tf.signal.stft(audio, frame_length=255, frame_step=128)
  spectrogram = tf.abs(spectrogram)
  spectrogram = spectrogram[..., tf.newaxis]
  spectrograms = []
  step = 2000
  start = 0
  end = step
  for i in range(np.shape(spectrogram)[0]//step):
    spectrograms.append(tf.image.resize(spectrogram[start:end,...], size = (step, 129)))
    start += step
    end += step
  return spectrograms

In [ ]:
def predict_voice(voice):
  spects = Voice_spec(voice)
  predictions = []
  for i in range(len(spects)):
    spec = spects[i][tf.newaxis,...]
    predictions.append( model.predict(spec, verbose=0) )
  prediction = np.mean(predictions, axis=0)
  return np.argmax(prediction)

In [ ]:
def soft_predict_voice(voice):
  spects = Voice_spec(voice)
  predictions = []
  for i in range(len(spects)):
    spec = spects[i][tf.newaxis,...]
    predictions.append( np.argmax(model.predict(spec, verbose=0)))
  prediction = np.bincount(predictions)
  return prediction

In [ ]:
for file in files:
  if file.endswith(".wav"):
    print(f"File Name: {file}")
    prediction = predict_voice(audios[file])
    print(f"Prediction: {class_names[prediction]}")
    print("-"*80)

File Name: Music Country [ musicmedia.ir ] 1.wav
Prediction: Disco
--------------------------------------------------------------------------------
File Name: 11. TCHAIKOVSKY- PIANO CONCERTO NO.1 OP.23- i. Allegro non troppo e molto maestoso.wav
Prediction: Classical
--------------------------------------------------------------------------------
File Name: 026_A Love_Like_This.wav
Prediction: Pop
--------------------------------------------------------------------------------
File Name: 18. SAINT-SAENS- SYMPHONY NO.3 IN C MINOR 'ORGAN' OP.78 (maestoso-allegro).wav
Prediction: Classical
--------------------------------------------------------------------------------
File Name: 027_Adele_Someone_Like_You.wav
Prediction: Country
--------------------------------------------------------------------------------
File Name: Imagine Dragons - Believer [128].wav
Prediction: Pop
--------------------------------------------------------------------------------
File Name: taylor_swift-_i_knew_you_w

In [ ]:
for file in files:
  if file.endswith(".wav"):
    print(f"File Name: {file}")
    predictions = soft_predict_voice(audios[file])
    preds = np.squeeze(predictions)
    sum = np.sum(preds)
    for i in range(len(preds)):
      if preds[i] != 0:
        print(f"{class_names[i]}: ", f"{round(preds[i]/sum*100, 2)}%")
    print("-"*80)

File Name: Music Country [ musicmedia.ir ] 1.wav
Disco:  100.0%
--------------------------------------------------------------------------------
File Name: 11. TCHAIKOVSKY- PIANO CONCERTO NO.1 OP.23- i. Allegro non troppo e molto maestoso.wav
Classical:  76.47%
Jazz:  23.53%
--------------------------------------------------------------------------------
File Name: 026_A Love_Like_This.wav
Classical:  5.0%
Country:  10.0%
Disco:  20.0%
Jazz:  5.0%
Pop:  50.0%
Reggae:  10.0%
--------------------------------------------------------------------------------
File Name: 18. SAINT-SAENS- SYMPHONY NO.3 IN C MINOR 'ORGAN' OP.78 (maestoso-allegro).wav
Classical:  100.0%
--------------------------------------------------------------------------------
File Name: 027_Adele_Someone_Like_You.wav
Classical:  4.17%
Country:  41.67%
Jazz:  20.83%
Pop:  20.83%
Rock:  12.5%
--------------------------------------------------------------------------------
File Name: Imagine Dragons - Believer [128].wav
Coun